In [ ]:
import numpy as np
import os
import psi4
import subprocess

In [ ]:
cwd = os.getcwd()
vibrate_dir = f"{cwd}/../VIBRATION"
xyz_dir = f"{cwd}/../XYZ"
os.chdir(vibrate_dir)

molecule_name = "ethanol"
xyz_file = f"{molecule_name}.xyz"
key_file = f"{molecule_name}.key"
prm_file = "amoeba09.prm"

In [ ]:
# Copy xyz, key, and prm files

os.system(f"cp {xyz_dir}/{xyz_file} .")
os.system(f"cp {xyz_dir}/{key_file} .")
os.system(f"cp {xyz_dir}/{prm_file} .")

In [ ]:
# Get coordinate

# Open and read the file
with open(xyz_file, "r") as f:
    lines = f.readlines()

# Extract atomic data (ignoring first two lines)
atomic_data = []
for line in lines[1:]:  # Skip first two lines
    parts = line.split()
    if len(parts) >= 4:  # Ensure the line contains atomic data
        element = parts[1]  # Atomic symbol
        x, y, z = parts[2], parts[3], parts[4]  # Coordinates
        atomic_data.append(f"{element}  {float(x):10.6f}  {float(y):10.6f}  {float(z):10.6f}")

# Print extracted data
atom_txt = ""
for atom in atomic_data:
    atom_txt += atom + "\n"

In [ ]:
# QM setting

method = "mp2"
basis = "aug-cc-pVTZ"
psi4.core.set_output_file('output.dat', False)
psi4.set_memory('10 GB')
psi4.core.set_num_threads(10)

psi4.set_options({
    'basis': basis,
    'd_convergence': 10,
    "PROPERTIES_ORIGIN": ["COM"],
})

In [ ]:
# Define molecule

molecule_structure = f"""
0 1
{atom_txt}
no_reorient
symmetry c1
no_com
units ang
"""

psi4.geometry(molecule_structure)

In [ ]:
# Compute vibrational frequency

energy, wfn = psi4.frequency(method, return_wfn=True)

In [ ]:
# Get vibrational frequency (eigenvalue and eigenvector)

# frequencies = wfn.frequency_analysis['omega'].data
# modes = wfn.frequency_analysis['q'].data
# modes = [m*psi4.constants.bohr2angstroms for m in modes]